In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

IMG_SIZE = 512

def load_images_from_folder(folder_path, label, img_size):
    data = []
    labels = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = load_img(img_path, target_size=(img_size, img_size))
        img_array = img_to_array(img) / 255.0
        data.append(img_array)
        labels.append(label)
    return data, labels

# Load training data
train_is_path = "ice_cream/train/ice_cream"
train_not_path = "ice_cream/train/non_ice_cream"
train_data_is, labels_is = load_images_from_folder(train_is_path, 1, IMG_SIZE)
train_data_not, labels_not = load_images_from_folder(train_not_path, 0, IMG_SIZE)

X_train = np.array(train_data_is + train_data_not)
y_train = np.array(labels_is + labels_not)

# Load testing data
test_is_path = "ice_cream/test/ice_cream"
test_not_path = "ice_cream/test/non_ice_cream"
test_data_is, test_labels_is = load_images_from_folder(test_is_path, 1, IMG_SIZE)
test_data_not, test_labels_not = load_images_from_folder(test_not_path, 0, IMG_SIZE)

X_test = np.array(test_data_is + test_data_not)
y_test = np.array(test_labels_is + test_labels_not)


In [ ]:
from sklearn.utils import shuffle

# Træning:
print("x_test: ", X_test.shape)
print("y_test: ", y_test.shape)
print("x_train: ", X_train.shape)
print("y_train: ", y_train.shape)

# Shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)


# CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binær classification (is / not-is)

model.compile(optimizer='adam',
              loss='binary_crossentropy', # also hvad pokker er det her
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
model.summary()



x_test:  (200, 512, 512, 3)
y_test:  (200,)
x_train:  (1800, 512, 512, 3)
y_train:  (1800,)
Epoch 1/10
 3/57 ━━━━━━━━━━━━━━━━━━━━ 3:14 4s/step - accuracy: 0.4583 - loss: 6.0873